In [76]:
'''
AUTHOR: Shawn Macburnie

All Tests that need to be run can be found at the bottom of the page with the function name starting with Test.

What you want to do is run every cell of code into memory and then once that is done you can can 
call the Test functions, or any other function you wish.
'''

'\nAUTHOR: Shawn Macburnie\n\nAll Tests that need to be run can be found at the bottom of the page with the function name starting with Test.\n\nWhat you want to do is run every cell of code into memory and then once that is done you can can \ncall the Test functions, or any other function you wish.\n'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator
import matplotlib.animation as animation
import copy
import theano
import theano.tensor as T
import csv
from random import randint
import os
import sys

In [4]:
#calculate Error from training data
def calculateError(w1, w2, data = [],display = True):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)
    
    if display:
        print("With: w1 = %f and w2 = %f"%(w1,w2))
    

    sumError = 0
    sumErrorSquare = 0
    if(not len(data)):
        with open('Data.txt') as fp:
            reader = csv.reader(fp, delimiter=',')
            for line in reader:
                data.append([int(line[0]), float(line[1]), float(line[2])])
    if display:
        print('y\t\tf(x)\t\tE\t\tE^2')
    for i in range(0,len(data)):
        x1 = data[i][1]
        x2 = data[i][2]
        f = logistic([x1,x2],[w1,w2])
        e = data[i][0] - f
        e2 = e**2
        sumError += e
        sumErrorSquare += e2
        if display:
            print('%f\t%f\t%f\t%f'%(data[i][0],f,e,e2))
    if display:
        print("\nSum:\t\t\t\t%f\t%f"%(sumError, sumErrorSquare))
    return sumErrorSquare

In [5]:
#plot Data from Data.txt
def plotData(w1,w2):
    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    for i in range(0,len(data)):
        if data[i][0] == 1:
            plt.scatter(data[i][1],data[i][2],20, 'r')
        else:
            plt.scatter(data[i][1],data[i][2],20, 'y')
    plt.plot([0,w1],[0,w2])
    plt.show()

In [6]:
#attempt to find my w
def findW(plot = True, showError = True, epsilon = 0.1):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)

    gs = T.grad(s, w)
    dlogistic = theano.function([x,w], gs)
    

    w1 = np.random.uniform(-2,2,1)[0]
    w2 = np.random.uniform(-2,2,1)[0]
    print("Starting Values: w1 = %f and w2 = %f"%(w1,w2))

    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    sumErrorSquare = 1
    iterations = 0
    while sumErrorSquare > epsilon:
        sumError = 0
        sumErrorSquare = 0
        sumErrorDeriv = 0
        for i in range(0,len(data)):
            x1 = data[i][1]
            x2 = data[i][2]
            f = logistic([x1,x2],[w1,w2])
            e = data[i][0] - f
            e2 = e**2
            sumError += e
            sumErrorSquare = sumErrorSquare + e2
            sumErrorDeriv -= 2 * e * dlogistic([x1,x2],[w1,w2])
        if iterations > 200:
            print("either The Data given is unsolvable or you have reached a local minima ... ")
            return calculateError(w1,w2, showError)
        iterations += 1
        w1 = w1 -  0.1 * sumErrorDeriv[0]
        w2 = w2 - 0.1 * sumErrorDeriv[1]
    print("Final Values: w1 = %f and w2 = %f"%(w1,w2))
    print("iterations took:",iterations)
    if plot:
        plotData(w1,w2)
    if showError:    
        print("\nCalculating Error ... ")
    return calculateError(w1,w2, showError)
    

In [7]:
#used to generate a gride and then calculate the error
def plotErrorSurface(numberOfEach = 20, data = []):
    dataPoints = generateGraphData(numberOfEach)
    xHolder = dataPoints[0]
    yHolder = dataPoints[1]
    z = np.array([calculateError(x,y, data,False) for x,y in zip(np.ravel(xHolder), np.ravel(yHolder))])
    Z = z.reshape(xHolder.shape)
    dataPoints.extend([Z])
    return dataPoints

In [8]:
#used to genereate range of values for w1 and w2
def generateGraphData(numberOfEach):
    x = np.linspace(-10,10,numberOfEach)
    #x = np.repeat(x, numberOfEach)
    y = np.linspace(-10,10,numberOfEach)
    #y = np.tile(y,numberOfEach)
    x, y = np.meshgrid(x,y)
    return [x,y]

In [9]:
#testing validity of the plotErrorSurface  function and generateGraphData
def TestErrorCalculation():
    print("The weight vectors in the graph all have an error of less then 0.005\n")
    data = plotErrorSurface(10)
    goodData = []
    for x,y,z in zip(np.ravel(data[0]), np.ravel(data[1]), np.ravel(data[2])):
        if z < 0.005:
            goodData.extend([[x, y]])
            plotData(x,y)
    print(goodData)

In [10]:
#testing functions to plot Error data
#NOTE: this may take a bit of time to run as it is generating alot of data
def Test3DErrorPlot():
    data = plotErrorSurface(10)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(data[0], data[1],data[2],rstride=1, cstride=1, cmap=cm.coolwarm)

    ax.set_xlabel('W1 Axis')
    ax.set_ylabel('W2 Axis')
    ax.set_zlabel('Error Axis')

    plt.show()

In [11]:
#testing the the function to find a w
def TestFindWeights():
    for i in range(0,4):
        error = findW(False, False)
        print("The error for these values:",error)
        print("")

In [69]:
#Animated error surface with 1 changing point
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
dataFile = []
with open('Data.txt') as fp:
    reader = csv.reader(fp, delimiter=',')
    for line in reader:
        dataFile.append([int(line[0]), float(line[1]), float(line[2])])
changingIndex = randint(0,len(dataFile) -1)
dataFile[changingIndex][1] += 0.5
with open("Data.txt", "w") as fp:
    writer = csv.writer(fp, delimiter=',')
    for i in range(0, len(dataFile)):
        writer.writerow([dataFile[i][0],dataFile[i][1],dataFile[i][2]])
data = plotErrorSurface(5)
ax.plot_surface(data[0], data[1],data[2],rstride=1, cstride=1, cmap=cm.coolwarm)
iterations = 0
deltaX = 1
deltaY = -2

axarr = fig.add_subplot(122)
data = []
with open('Data.txt') as fp:
    reader = csv.reader(fp, delimiter=',')
    for line in reader:
        data.append([int(line[0]), float(line[1]), float(line[2])])
for i in range(0,len(data)):
    if data[i][0] == 1:
        axarr.scatter(data[i][1],data[i][2],20, 'r')
    else:
        axarr.scatter(data[i][1],data[i][2],20, 'y')
        
def animateError(i):
    global iterations, deltaX, deltaY,data
    iterations +=1
    data[changingIndex][1] += deltaX
    data[changingIndex][2] += deltaY
    if data[changingIndex][1] < -10:
        deltaX *= -1
    if data[changingIndex][1] > 10:
        deltaX *= -1
    if data[changingIndex][2] < -10:
        deltaY *= -1
    if data[changingIndex][2] > 10:
        deltaY *= -1
#     with open("Data.txt", "w") as fp:
#         writer = csv.writer(fp, delimiter=',')
#         for i in range(0, len(dataFile)):
#             writer.writerow([dataFile[i][0],dataFile[i][1],dataFile[i][2]])
    ax.clear()
    errorSurface = plotErrorSurface(5, data)
    ax.plot_surface(errorSurface[0], errorSurface[1],errorSurface[2],rstride=1, cstride=1, cmap=cm.coolwarm)
    ax.set_xlabel('W1 Axis')
    ax.set_ylabel('W2 Axis')
    ax.set_zlabel('Error Axis; iteration:%d'%i,)
    axarr.clear()
    for i in range(0,len(data)):
        if data[i][0] == 1:
            axarr.scatter(data[i][1],data[i][2],20, 'r')
        else:
            axarr.scatter(data[i][1],data[i][2],20, 'y')
    
      
ani1 = animation.FuncAnimation(fig,animateError, interval=1000)

ax.set_xlabel('W1 Axis')
ax.set_ylabel('W2 Axis')
ax.set_zlabel('Error Axis; iteration:%d'%iterations,)

plt.show()

In [22]:
#offLine animated error plot surface
# you must have run calcOfflineData(0.25,-0.5,40, 40) with what I suggest those parameters, it will take a long time
# to run but it will give the best results. If not and you already have a offlineErrorData.txt file then you
# will not need to run that function
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(121, projection='3d')
deltaX = 0
deltaY = 0
iteration = 0
changeIndex = 0
meshNumber = 0
errorData = []

with open('offlineErrorData.txt') as fp:
    reader = csv.reader(fp, delimiter=',')
    for line in reader:
        if len(line) == 5:
            deltaX, deltaY, changeIndex, iteration, meshNumber = [float(x) for x in line]
        else:
            errorData.append([float(x) for x in line])
iteration = int(iteration)
changeIndex = int(changeIndex)
axarr = fig.add_subplot(122)
data = []
with open('Data.txt') as fp:
    reader = csv.reader(fp, delimiter=',')
    for line in reader:
        data.append([int(line[0]), float(line[1]), float(line[2])])
for i in range(0,len(data)):
    if data[i][0] == 1:
        axarr.scatter(data[i][1],data[i][2],20, 'r')
    else:
        axarr.scatter(data[i][1],data[i][2],20, 'y')

meshx = np.linspace(-10,10,meshNumber)
meshy = np.linspace(-10,10,meshNumber)
meshx, meshy = np.meshgrid(meshx,meshy)      
def animateError(i):
    global deltaX, deltaY,data, iteration, changeIndex
    axarr.clear()
    for x in range(0,len(data)):
        if data[x][0] == 1:
            axarr.scatter(data[x][1],data[x][2],20, 'r')
        else:
            axarr.scatter(data[x][1],data[x][2],20, 'y')
    data[changeIndex][1] += deltaX
    data[changeIndex][2] += deltaY
    if data[changeIndex][1] < -10:
        deltaX *= -1
    if data[changeIndex][1] > 10:
        deltaX *= -1
    if data[changeIndex][2] < -10:
        deltaY *= -1
    if data[changeIndex][2] > 10:
        deltaY *= -1
    ax.clear()
    errorSurface = np.asarray(errorData[i % iteration ]).reshape(meshx.shape)
    ax.plot_surface(meshx, meshy,errorSurface,rstride=1, cstride=1, cmap=cm.coolwarm)
    ax.set_xlabel('W1 Axis')
    ax.set_ylabel('W2 Axis')
    ax.set_zlabel('Error Axis; iteration:%d'%i,)
    
      
ani1 = animation.FuncAnimation(fig,animateError, interval=1000)

ax.set_xlabel('W1 Axis')
ax.set_ylabel('W2 Axis')
ax.set_zlabel('Error Axis')

plt.show()

In [34]:
#you must have a file called Data.txt where it will be your training data will be saved to. suggest using my data 
# generation files in my format.
def calcOfflineData(deltaX = 1, deltaY = -2,meshNumber = 10,iterations = 10, fileName = "offlineErrorData.txt"):
    data = []
    x = np.linspace(-10,10,meshNumber)
    y = np.linspace(-10,10,meshNumber)
    x, y = np.meshgrid(x,y)
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    
    changingIndex = randint(0,len(data) -1)
    
    dataFile = open(fileName, "w")
    writer = csv.writer(dataFile, delimiter=',')
    writer.writerow([deltaX, deltaY,changingIndex, iterations, meshNumber])
    
    for i in range(0,iterations):
        print("On Iteration %d of moving points"%i)
        sys.stdout.flush()
        data[changingIndex][1] += deltaX
        data[changingIndex][2] += deltaY
        if data[changingIndex][1] < -10:
            deltaX *= -1
        if data[changingIndex][1] > 10:
            deltaX *= -1
        if data[changingIndex][2] < -10:
            deltaY *= -1
        if data[changingIndex][2] > 10:
            deltaY *= -1
        counter = 0
        totalIter = meshNumber**2
        print("Done(out of %d):"%totalIter, end="")
        zData = []
        for X,Y in zip(np.ravel(x), np.ravel(y)):
            zData.extend([calculateError(X,Y, data,False)])
            if counter % int((totalIter / 10)) == 0:
                print(" %d"%counter, end="")
            counter += 1
        z = np.array(zData)
        print('\n', end="")
        writer.writerow(z)
    dataFile.close()
    print("Done Offline Calculations!")
            

In [36]:
calcOfflineData(0.25,-0.5,40, 40)

On Iteration 0 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 1 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 2 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 3 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 4 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 5 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 6 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 7 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 8 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 9 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 1120 1280 1440
On Iteration 10 of moving points
Done(out of 1600): 0 160 320 480 640 800 960 11

In [16]:
deltaX = 0
deltaY = 0
iteration = 0
changeIndex = 0
meshNumber = 0
errorData = []

with open('offlineErrorData.txt') as fp:
    reader = csv.reader(fp, delimiter=',')
    for line in reader:
        if len(line) == 5:
            deltaX, deltaY, changeIndex, iteration, meshNumber = [float(x) for x in line]
print(changeIndex)

27.0


In [1]:
print(hey)

NameError: name 'hey' is not defined